In [1]:
import os

import ir_datasets
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm


os.environ["IR_DATASETS_HOME"] = "/storage/kliffeup/kliffeup/allknower_data"

In [ ]:
DATASET_PREFIX = "wikir"
DATASET_VERSION = "en78k"
STAGE = "training"

In [2]:
wikir_dataset = ir_datasets.load(f"{DATASET_PREFIX}/{DATASET_VERSION}/{STAGE}")

In [3]:
wikir_dataset.docs_cls()._fields

('doc_id', 'text')

In [4]:
for doc in wikir_dataset.docs_iter()[:2]:
    print(doc)

GenericDoc(doc_id='1781133', text='it was used in landing craft during world war ii and is used today in private boats and training facilities the 6 71 is an inline six cylinder diesel engine the 71 refers to the displacement in cubic inches of each cylinder the firing order of the engine is 1 5 3 6 2 4 the engine s compression ratio is 18 7 1 with a 4 250 inch bore and a 5 00 inch stroke the engine weighs and is 54 inches long 29 inches wide and 41 inches tall at 2 100 revolutions per minute the engine is capable of producing 230 horse power 172 kilowatts v type versions of the 71 series were developed in 1957 the 6 71 is a two stroke engine as the engine will not naturally aspirate air is provided via a roots type blower however on the 6 71t models a turbocharger and a supercharger are utilized fuel is provided by unit injectors one per cylinder the amount of fuel injected into the engine is controlled by the engine s governor the engine cooling is via liquid in a water jacket in a b

In [5]:
query_idx = -1

for query in wikir_dataset.queries_iter():
    query_idx += 1
    if query_idx >= 5:
        break
    print(query)

GenericQuery(query_id='123839', text='yanni')
GenericQuery(query_id='188629', text='k pop')
GenericQuery(query_id='13898', text='venice film festival')
GenericQuery(query_id='316959', text='downtown brooklyn')
GenericQuery(query_id='515031', text='pennsylvania house of representatives')


In [6]:
qrel_idx = -1

for qrel in wikir_dataset.qrels_iter():
    qrel_idx += 1
    if qrel_idx >= 5:
        break
    print(qrel)

TrecQrel(query_id='123839', doc_id='123839', relevance=2, iteration='0')
TrecQrel(query_id='123839', doc_id='1793430', relevance=1, iteration='0')
TrecQrel(query_id='123839', doc_id='806300', relevance=1, iteration='0')
TrecQrel(query_id='123839', doc_id='806075', relevance=1, iteration='0')
TrecQrel(query_id='123839', doc_id='836567', relevance=1, iteration='0')


In [7]:
bm25_score_idx = -1

for bm25_score in wikir_dataset.scoreddocs_iter():
    bm25_score_idx += 1
    if bm25_score_idx >= 5:
        break
    print(bm25_score)

GenericScoredDoc(query_id='123839', doc_id='806300', score=20.720094194011075)
GenericScoredDoc(query_id='123839', doc_id='123839', score=19.91782871489318)
GenericScoredDoc(query_id='123839', doc_id='836567', score=18.824522997710037)
GenericScoredDoc(query_id='123839', doc_id='806326', score=18.824522997710037)
GenericScoredDoc(query_id='123839', doc_id='806075', score=17.246712972547066)


# Feature engineering

In [8]:
MAX_RELEVANCE = 2

In [9]:
queries = pd.DataFrame(
    sorted(
        map(
            lambda item: (int(item.query_id), item.text),
            wikir_dataset.queries_iter()
        )  
    ),
    columns=["query_id", "query"],
)

In [10]:
docs = pd.DataFrame(
    sorted(
        map(
            lambda item: (int(item.doc_id), item.text),
            wikir_dataset.docs_iter()
        ),
        key=lambda item: item[0]
    ),
    columns=["doc_id", "doc"],
)

In [11]:
query_id_to_doc_id_rels = pd.DataFrame(
    sorted(
        map(
            lambda item: (int(item.query_id), int(item.doc_id), item.relevance / MAX_RELEVANCE),
            wikir_dataset.qrels_iter()
        ),
        key=lambda item: (item[0], item[1])
    ),
    columns=["query_id", "doc_id", "relevance"],
)

In [12]:
query_id_to_doc_id_bm25_scores = pd.DataFrame(
    sorted(
        map(
            lambda item: (int(item.query_id), int(item.doc_id), item.score),
            wikir_dataset.scoreddocs_iter()
        ),
        key=lambda item: (item[0], item[1])
    ),
    columns=["query_id", "doc_id", "bm25_score"],    
)

In [13]:
data = query_id_to_doc_id_bm25_scores\
.merge(query_id_to_doc_id_rels, how="outer")\
.merge(queries, how="left")\
.merge(docs, how="left")

In [14]:
data

,query_id,doc_id,bm25_score,relevance,query,doc
0,79,79,16.106074,1.0,actinopterygii,these actinopterygian fin rays attach directly...
1,79,12139,0.000000,NaN,actinopterygii,even the acts of launching discharging artille...
2,79,34714,0.000000,NaN,actinopterygii,state of nebraska as of the 2010 united states...
3,79,57100,0.000000,NaN,actinopterygii,as of the 2010 united states census the cdp s ...
4,79,92466,0.000000,NaN,actinopterygii,its lyrics were written by the buenos aires bo...
...,...,...,...,...,...,...
185921,2433785,2433250,13.206970,0.5,dhanbad sadar subdivision,one of the many spurs of pareshnath hill 1 365...
185922,2433785,2433522,15.420900,0.5,dhanbad sadar subdivision,the damodar river the most important river of ...
185923,2433785,2433549,9.231121,0.5,dhanbad sadar subdivision,while the damodar flows along the southern bou...
185924,2433785,2433785,37.233094,1.0,dhanbad sadar subdivision,initially the district was split into two subd...


# Recover BM25 scores

In [15]:
import bm25s
import Stemmer

stemmer = Stemmer.Stemmer("english")

/storage/kliffeup/conda_envs/allknower/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [16]:
for _, row in docs.iterrows():
    print(dict(row))
    break

{'doc_id': 5, 'doc': 'he was the son of the nereid thetis and peleus king of phthia achilles most notable feat during the trojan war was the slaying of the trojan prince hector outside the gates of troy although the death of achilles is not presented in the iliad other sources concur that he was killed near the end of the trojan war by paris who shot him in the heel with an arrow later legends beginning with statius unfinished epic achilleid written in the 1st century ad state that achilles was invulnerable in all of his body except for his heel because when his mother thetis dipped him in the river styx as an infant she held him by one of his heels alluding to these legends the term achilles heel has come to mean a point of weakness especially in someone or something with an otherwise strong constitution the achilles tendon is also named after him due to these legends linear b tablets attest to the personal name achilleus in the forms a ki re u and a ki re we the latter being the dati

# Calc scores over whole doc corpus

In [17]:
corpus_json = docs.apply(lambda row: dict(row), axis=1).values.tolist()

In [18]:
corpus_text = list(map(lambda item: item["doc"], corpus_json))

In [19]:
tokenizer = bm25s.tokenization.Tokenizer(
    stemmer=stemmer,
    lower=True, # lowercase the tokens
    stopwords="english",  # or pass a list of stopwords
    splitter=r"\w+",  # by default r"(?u)\b\w\w+\b", can also be a function
)

In [20]:
# Tokenize the corpus
corpus_tokens = tokenizer.tokenize(
    corpus_text, 
    update_vocab=True, # update the vocab as we tokenize
    return_as="ids"
)

# corpus_tokens = bm25s.tokenize(corpus_text, 
#                                # stopwords="en", stemmer=stemmer
#                               )

In [21]:
retriever = bm25s.BM25(corpus=corpus_json)

In [22]:
retriever.index(corpus_tokens)

In [23]:
query = "actinopterygii"

In [24]:
query_tokens = tokenizer.tokenize(
    [query], 
    update_vocab=False
)

# query_tokens = bm25s.tokenize(query,
#                               stopwords="en", stemmer=stemmer
#                              )

In [25]:
# query_tokens

[[850]]

In [26]:
# docs_79_query = data[data["query_id"] == 79][["doc_id", "doc"]]

In [27]:
# corpus_79_json = docs_79_query.apply(lambda row: dict(row), axis=1).values.tolist()

In [28]:
# corpus_79_text = list(map(lambda item: item["doc"], corpus_79_json))

In [29]:
# corpus_79_tokens = tokenizer.tokenize(
#     corpus_79_text, 
#     update_vocab=False, # update the vocab as we tokenize
#     return_as="ids"
# )

In [30]:
# corpus_79_text_ids = list(map(lambda item: retriever.corpus.index(item), corpus_79_json))

In [31]:
# results, scores = retriever.retrieve(query_tokens, k=10, show_progress=False, return_as="tuple")

In [32]:
# data[data["query_id"] == 79][["doc_id"]].values.flatten().tolist()

# Calc BM25S scores

In [33]:
doc_scores = []

In [34]:
for _, row in tqdm(queries.iterrows()):
    query, query_id = row.query, row.query_id
    
    doc_ids_query = data[data["query_id"] == query_id][["doc_id"]].values.flatten().tolist()

    query_tokens = tokenizer.tokenize(
        [query], 
        update_vocab=False,
        show_progress=False,
    )
    
    results, scores = retriever.retrieve(query_tokens, k=len(corpus_json), show_progress=False)
    doc_ids_results = list(map(lambda item: item["doc_id"], results[0]))
    doc_scores += [float(scores[0, doc_ids_results.index(doc_id)]) for doc_id in doc_ids_query]

1444it [06:48,  3.53it/s]


# Calc pos statistics

In [36]:
def calc_query_term_ngram_coverage(query_tokens: list[int], doc_tokens: list[int], n: int = 1, normalize: bool = False) -> int | float:
    if n > min(len(query_tokens), len(doc_tokens)):
        return 0

    query_ngram_counter = {}
    for i in range(len(query_tokens) - n + 1):
        query_ngram_counter[tuple(query_tokens[i:i + n])] = 0

    for i in range(len(doc_tokens) - n + 1):
        doc_ngram = tuple(doc_tokens[i:i + n])
        if doc_ngram in query_ngram_counter and query_ngram_counter[doc_ngram] == 0:
            query_ngram_counter[doc_ngram] = 1

    query_term_ngram_coverage = sum(query_ngram_counter.values())

    if normalize:
        query_term_ngram_coverage /= len(query_ngram_counter)
    
    return query_term_ngram_coverage

In [37]:
calc_query_term_ngram_coverage([1, 2, 3], [1, 2, 4, 2, 4], n=2, normalize=True)

0.5

In [38]:
def calc_doc_length(doc_tokens: list[int]) -> int:
    return len(doc_tokens)

In [39]:
def calc_query_length(query_tokens: list[int]) -> int:
    return len(query_tokens)

In [40]:
def calc_query_term_first_occurrence(query_tokens: list[int], doc_tokens: list[int], normalize: bool = False) -> int:
    first_occurrence = len(doc_tokens)
    
    for i, doc_token in enumerate(doc_tokens):
        if doc_token in query_tokens:
            first_occurrence = i
            break
     
    if normalize:
        first_occurrence /= len(doc_tokens)

    return first_occurrence

In [41]:
calc_query_term_first_occurrence([1, 2, 3], [4, 1, 4, 2, 4], normalize=True)

0.2

In [42]:
query_length = []
doc_length = []
query_term_unigram_coverage_unnormalized = []
query_term_bigram_coverage_unnormalized = []
query_term_trigram_coverage_unnormalized = []
query_term_first_occurrence_unnormalized = []
query_term_last_occurrence_unnormalized = []
span_length_unnormalized = []
query_term_unigram_coverage_normalized = []
query_term_bigram_coverage_normalized = []
query_term_trigram_coverage_normalized = []
query_term_first_occurrence_normalized = []
query_term_last_occurrence_normalized = []
span_length_normalized_doc = []
span_length_normalized_query = []

In [43]:
doc_ids = docs["doc_id"].values.tolist()

In [44]:
for _, row in tqdm(queries.iterrows()):
    query, query_id = row.query, row.query_id
    
    doc_ids_query = data[data["query_id"] == query_id][["doc_id"]].values.flatten().tolist()

    query_tokens = tokenizer.tokenize(
        [query], 
        update_vocab=False,
        show_progress=False,
    )[0]

    docs_tokens_query = list(map(lambda doc_id: corpus_tokens[doc_ids.index(doc_id)], doc_ids_query))
    # docs_tokens_query = [corpus_tokens[doc_ids.index(doc_id)] for doc_id in doc_ids_query]
    query_length += [calc_query_length(query_tokens)] * len(doc_ids_query)

    for doc_tokens in docs_tokens_query:
        doc_length.append(calc_doc_length(doc_tokens))
        query_term_unigram_coverage_unnormalized.append(calc_query_term_ngram_coverage(query_tokens, doc_tokens, n=1))
        query_term_bigram_coverage_unnormalized.append(calc_query_term_ngram_coverage(query_tokens, doc_tokens, n=2))
        query_term_trigram_coverage_unnormalized.append(calc_query_term_ngram_coverage(query_tokens, doc_tokens, n=3))
        query_term_first_occurrence_unnormalized.append(calc_query_term_first_occurrence(query_tokens, doc_tokens))
        query_term_last_occurrence_unnormalized.append((doc_length[-1] - 1) - calc_query_term_first_occurrence(query_tokens, doc_tokens[::-1]))
        span_length_unnormalized.append(query_term_last_occurrence_unnormalized[-1] - query_term_first_occurrence_unnormalized[-1] + 1)
        query_term_unigram_coverage_normalized.append(calc_query_term_ngram_coverage(query_tokens, doc_tokens, n=1, normalize=True))
        query_term_bigram_coverage_normalized.append(calc_query_term_ngram_coverage(query_tokens, doc_tokens, n=2, normalize=True))
        query_term_trigram_coverage_normalized.append(calc_query_term_ngram_coverage(query_tokens, doc_tokens, n=3, normalize=True))
        query_term_first_occurrence_normalized.append(calc_query_term_first_occurrence(query_tokens, doc_tokens, normalize=True))
        query_term_last_occurrence_normalized.append(query_term_last_occurrence_unnormalized[-1] / doc_length[-1])
        span_length_normalized_doc.append(span_length_unnormalized[-1] / doc_length[-1])
        if query_length[-1]:
            span_length_normalized_query.append(span_length_unnormalized[-1] / query_length[-1])
        else:
            span_length_normalized_query.append(0)

1444it [05:41,  4.23it/s]


In [45]:
(
    len(doc_scores),
    len(query_length), 
    len(doc_length), 
    len(query_term_unigram_coverage_unnormalized),
    len(query_term_bigram_coverage_unnormalized),
    len(query_term_trigram_coverage_unnormalized),
    len(query_term_first_occurrence_unnormalized),
    len(query_term_last_occurrence_unnormalized),
    len(span_length_unnormalized),
    len(query_term_unigram_coverage_normalized),
    len(query_term_bigram_coverage_normalized),
    len(query_term_trigram_coverage_normalized),
    len(query_term_first_occurrence_normalized),
    len(query_term_last_occurrence_normalized),
    len(span_length_normalized_doc),
    len(span_length_normalized_query),
)

(185926,
 185926,
 185926,
 185926,
 185926,
 185926,
 185926,
 185926,
 185926,
 185926,
 185926,
 185926,
 185926,
 185926,
 185926,
 185926)

In [46]:
data["bm25s_score"] = doc_scores
data["query_length"] = query_length
data["doc_length"] = doc_length
data["query_term_unigram_coverage_unnormalized"] = query_term_unigram_coverage_unnormalized
data["query_term_bigram_coverage_unnormalized"] = query_term_bigram_coverage_unnormalized
data["query_term_trigram_coverage_unnormalized"] = query_term_trigram_coverage_unnormalized
data["query_term_first_occurrence_unnormalized"] = query_term_first_occurrence_unnormalized
data["query_term_last_occurrence_unnormalized"] = query_term_last_occurrence_unnormalized
data["span_length_unnormalized"] = span_length_unnormalized
data["query_term_unigram_coverage_normalized"] = query_term_unigram_coverage_normalized
data["query_term_bigram_coverage_normalized"] = query_term_bigram_coverage_normalized
data["query_term_trigram_coverage_normalized"] = query_term_trigram_coverage_normalized
data["query_term_first_occurrence_normalized"] = query_term_first_occurrence_normalized
data["query_term_last_occurrence_normalized"] = query_term_last_occurrence_normalized
data["span_length_normalized_doc"] = span_length_normalized_doc
data["span_length_normalized_query"] = span_length_normalized_query

In [47]:
data.head()

,query_id,doc_id,bm25_score,relevance,query,doc,bm25s_score,query_length,doc_length,query_term_unigram_coverage_unnormalized,...,query_term_first_occurrence_unnormalized,query_term_last_occurrence_unnormalized,span_length_unnormalized,query_term_unigram_coverage_normalized,query_term_bigram_coverage_normalized,query_term_trigram_coverage_normalized,query_term_first_occurrence_normalized,query_term_last_occurrence_normalized,span_length_normalized_doc,span_length_normalized_query
0,79,79,16.106074,1.0,actinopterygii,these actinopterygian fin rays attach directly...,6.407508,1,139,1,...,88,133,46,1.0,0.0,0.0,0.633094,0.956835,0.330935,46.0
1,79,12139,0.000000,NaN,actinopterygii,even the acts of launching discharging artille...,0.000000,1,133,0,...,133,-1,-133,0.0,0.0,0.0,1.000000,-0.007519,-1.000000,-133.0
2,79,34714,0.000000,NaN,actinopterygii,state of nebraska as of the 2010 united states...,0.000000,1,135,0,...,135,-1,-135,0.0,0.0,0.0,1.000000,-0.007407,-1.000000,-135.0
3,79,57100,0.000000,NaN,actinopterygii,as of the 2010 united states census the cdp s ...,0.000000,1,146,0,...,146,-1,-146,0.0,0.0,0.0,1.000000,-0.006849,-1.000000,-146.0
4,79,92466,0.000000,NaN,actinopterygii,its lyrics were written by the buenos aires bo...,0.000000,1,135,0,...,135,-1,-135,0.0,0.0,0.0,1.000000,-0.007407,-1.000000,-135.0


In [48]:
data = data.fillna(0)

In [49]:
data["doc_length_scaled"] = data.groupby("query_id")["doc_length"].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
# df['Z_Score_Normalized'] = df.groupby('Group')['Value'].transform(lambda x: (x - x.mean()) / x.std())
# df['Mean_Normalized'] = df.groupby('Group')['Value'].transform(lambda x: (x - x.mean()) / (x.max() - x.min()))

In [53]:
# data.to_csv("wikir_en1k_training_preprocessed.csv", index=None)

In [51]:
data.columns

Index(['query_id', 'doc_id', 'bm25_score', 'relevance', 'query', 'doc',
       'bm25s_score', 'query_length', 'doc_length',
       'query_term_unigram_coverage_unnormalized',
       'query_term_bigram_coverage_unnormalized',
       'query_term_trigram_coverage_unnormalized',
       'query_term_first_occurrence_unnormalized',
       'query_term_last_occurrence_unnormalized', 'span_length_unnormalized',
       'query_term_unigram_coverage_normalized',
       'query_term_bigram_coverage_normalized',
       'query_term_trigram_coverage_normalized',
       'query_term_first_occurrence_normalized',
       'query_term_last_occurrence_normalized', 'span_length_normalized_doc',
       'span_length_normalized_query', 'doc_length_scaled'],
      dtype='object')

In [52]:
data[['query_id', 'doc_id', 'bm25_score', 'relevance',
       'bm25s_score', 'query_length', 'doc_length',
       'query_term_unigram_coverage_unnormalized',
       'query_term_bigram_coverage_unnormalized',
       'query_term_trigram_coverage_unnormalized',
       'query_term_first_occurrence_unnormalized',
       'query_term_last_occurrence_unnormalized', 'span_length_unnormalized',
       'query_term_unigram_coverage_normalized',
       'query_term_bigram_coverage_normalized',
       'query_term_trigram_coverage_normalized',
       'query_term_first_occurrence_normalized',
       'query_term_last_occurrence_normalized', 'span_length_normalized_doc',
       'span_length_normalized_query', 'doc_length_scaled']].to_csv(f"{DATASET_PREFIX}_{DATASET_VERSION}_{STAGE}_preprocessed_light.csv", index=None)